# Combine Labels

In [1]:
# import and constant
import os
import numpy as np
import nibabel as nib
from utils.nrrd2nii import nrrd2nii

source_path = "data/uwa_aaa"
combined_path = "data/uwa_aaa_combined"

os.makedirs(combined_path, exist_ok=True)

for patient_folder in sorted(os.listdir(source_path)):
    patient_path = os.path.join(source_path, patient_folder)
    aaa_label = None
    blood_label = None
    for root, dirs, files in os.walk(patient_path):
        for file in files:
            if file == "CT_AAA_label.nrrd":
                aaa_label = nrrd2nii(os.path.join(root, file)).get_fdata()

                assert (
                    len(np.unique(aaa_label)) == 2
                ), f"Patient {patient_folder} label error"

                highest_label = np.max(aaa_label)
                aaa_label[aaa_label == highest_label] = 1

            if file == "CT_blood_label.nrrd":
                blood_label = nrrd2nii(os.path.join(root, file)).get_fdata()
                
                assert (
                    len(np.unique(blood_label)) == 2
                ), f"Patient {patient_folder} label error"

                highest_label = np.max(blood_label)
                blood_label[blood_label == highest_label] = 1

    if (aaa_label is None) or (blood_label is None):
        print(f"Patient {patient_folder} label missing")
        continue

    if aaa_label.shape != blood_label.shape:
        print(f"Patient {patient_folder} label shape error: {aaa_label.shape} vs {blood_label.shape}")
        continue
    
    if (blood_label.shape[0] * blood_label.shape[1] * blood_label.shape[2]) > 10000000:
        print(f"Patient {patient_folder} too large: {blood_label.shape}")
        continue

    combined_label = np.zeros_like(aaa_label)
    combined_label[aaa_label == 1] = 1
    combined_label[blood_label == 1] = 2
    
    # if shape contains singular, make it even
    padding = []
    for dim in combined_label.shape:
        if dim % 2 == 1:
            padding.append(1)
        else:
            padding.append(0)
    combined_label = np.pad(
        combined_label,
        ((0, padding[0]), (0, padding[1]), (0, padding[2])),
        mode="constant",
    )
    
    label_nii = nib.Nifti1Image(combined_label, np.eye(4))
    nib.save(label_nii, os.path.join(combined_path, f"{patient_folder}_combined.nii.gz"))

Patient Patient19 too large: (231, 292, 430)
Patient Patient2 label shape error: (138, 123, 151) vs (138, 123, 200)
Patient Patient21 label missing
Patient Patient22 label missing
Patient Patient23 label missing
Patient Patient24 label missing
Patient Patient25 label missing
Patient Patient27 label missing
Patient Patient28 label missing
Patient Patient29 label missing
Patient Patient30 label missing
Patient Patient31 label missing
Patient Patient4 label shape error: (152, 157, 171) vs (152, 157, 95)
Patient Patient7 label shape error: (126, 135, 147) vs (512, 512, 239)


# Inspeact labels

In [3]:
from utils.check_image import check_image
import nibabel as nib
from utils.nrrd2nii import nrrd2nii

image = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_cropped.nrrd").get_fdata()
label = nib.load("data/uwa_aaa_combined/Patient1_combined.nii.gz").get_fdata()
label0 = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_AAA_label.nrrd").get_fdata()
label1 = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_blood_label.nrrd").get_fdata()

In [ ]:
check_image(
    [
        {"image": image, "title": "image", "is_label": False},
        {"image": label0, "title": "label", "is_label": True},
        {"image": label1, "title": "label", "is_label": True},
        {"image": label, "title": "combined", "is_label": True},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=125), IntSlider(value=0, description…